### チュートリアルの学習
- https://yutaroogawa.github.io/pytorch_tutorials_jp/

### Tensor

In [32]:
# -*- coding: utf-8 -*-
import numpy as np

# N：バッチサイズ         D_in：入力層の次元数
# H：隠れ層の次元数       D_out： 出力層の次元数
N, D_in, H, D_out = 64, 1000, 100, 10

# 乱数により入力データと目標となる出力データを生成
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# 乱数による重みの初期化
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6
for t in range(500):
    # 順伝播： 予測値yの計算
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)

    # 損失の計算と表示
    loss = np.square(y_pred - y).sum()
    if t % 100 == 99:
        print(t, loss)

    # 逆伝搬：損失に対するW1とw2の勾配の計算
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    # 重みの更新
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

99 442.7127760849192
199 2.4212810292177798
299 0.026646992524446286
399 0.0003348938253444464
499 4.357874504552627e-06


In [33]:
#日本語訳追記：
#学習前の重みを定義（前の学習で使用された重みの初期値とは異なります）
w1_unlearned = np.random.randn(D_in, H)
w2_unlearned = np.random.randn(H, D_out)

#学習前の出力を算出
h = x.dot(w1_unlearned)
h_relu = np.maximum(h, 0)
y_pred = h_relu.dot(w2_unlearned)
print(f"学習前出力：{np.round(y_pred[0], decimals=2)}")

#学習によって得られた重みで出力を算出
h = x.dot(w1)
h_relu = np.maximum(h, 0)
y_pred = h_relu.dot(w2)
print(f"学習後出力：{np.round(y_pred[0], decimals=2)}")

#目的の出力yとの比較
print(f"目的の出力：{np.round(y[0], decimals=2)}")


学習前出力：[-164.37    9.64   34.79  510.17    4.69 -316.84   12.82  201.45 -273.12
  555.25]
学習後出力：[-0.73  0.83  0.05  0.35 -0.35  0.48 -0.15  0.47  1.06  0.65]
目的の出力：[-0.73  0.83  0.05  0.35 -0.35  0.48 -0.15  0.47  1.06  0.65]


### PyTorch: Tensors

In [35]:
# -*- coding: utf-8 -*-
import torch

dtype = torch.float
device = torch.device("mps")

# N：バッチサイズ         D_in：入力層の次元数
# H：隠れ層の次元数       D_out： 出力層の次元数
N, D_in, H, D_out = 64, 1000, 100, 10

# 乱数により入力データと目標となる出力データを生成
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 乱数による重みの初期化
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    # 順伝播： 予測値yの計算
    # mm() は行列の掛け算
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    # x * w1 * w2 -> 予測値 y^
    y_pred = h_relu.mm(w2)

    # 損失の計算と表示
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

   # 逆伝搬：損失に対するW1とw2の勾配の計算
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # 確率的勾配降下法による重みの更新
    # grad は torch 同志をかけた時に生じる（？）
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

99 710.273193359375
199 4.414570331573486
299 0.03892833739519119
399 0.000632779614534229
499 7.131210441002622e-05


### Autgrad

In [45]:
# -*- coding: utf-8 -*-
import torch

dtype = torch.float
device = torch.device("mps")

# N：バッチサイズ         D_in：入力層の次元数
# H：隠れ層の次元数       D_out： 出力層の次元数
N, D_in, H, D_out = 64, 1000, 100, 10

# 日本語訳注：デフォルトでは requires_grad は False が設定されています。
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 乱数による重みを表すTensorの定義
# 逆伝播の際、このTensorに対する勾配を計算する場合は、requires_grad=Trueを指定します
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # 順伝播： Tensorの演算を利用して予測結果yの算出
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    # Tensorを用いた損失の計算と表示
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # autograd を使用して逆伝播の計算をします。
    # backward()によりrequires_gradにTrueが設定されているすべてのTensorに対して、
    # 損失の勾配を計算を行います。これにより、w1.gradとw2.gradはそれぞれw1とw2に
    # 対する損失の勾配を保持するTensorになります。
    loss.backward()

    # 確率的勾配降下法を使って手動で重みを更新します。
    # 別の方法としては、weight.dataとweight.grad.dataを利用する方法があります。
    # tensor.dataはtensorと保存領域を共有するTensorを返しますが、履歴は追跡されません。
    # torch.optim.SGDを利用して追跡を回避することも可能です。
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # 重みの更新後、手動で勾配を0に初期化
        w1.grad.zero_()
        w2.grad.zero_()

99 862.5645751953125
tensor([[ 3.6285e+01, -8.8487e+00,  1.5507e+01,  ..., -2.6105e+01,
         -1.5346e+00, -1.3366e+01],
        [ 5.6110e+01, -3.1637e+01, -1.3721e+01,  ..., -4.7835e+01,
          3.5215e+00, -8.2975e-01],
        [-1.7398e+01,  4.5638e+00, -1.1748e+01,  ..., -1.8335e+01,
         -7.6204e-01,  1.0017e+01],
        ...,
        [ 8.3582e+01, -3.8635e+01, -3.1670e+00,  ..., -3.0016e+01,
          5.5061e+00, -3.3824e+01],
        [ 3.1997e+01, -2.3670e+00,  5.4479e+00,  ..., -6.3684e+00,
         -1.5295e+00,  1.2324e+01],
        [ 1.8240e-02,  2.8511e+01,  2.8008e+00,  ..., -3.8512e+01,
          5.0114e+00, -1.9621e+01]], device='mps:0')
199 12.628241539001465
tensor([[ 1.6824e+00, -9.6306e-01,  1.4818e+00,  ...,  2.3631e+00,
          2.5596e-01, -1.1032e+00],
        [ 7.3589e+00, -3.2366e+00, -1.4048e-01,  ..., -7.9956e+00,
         -2.5128e-01, -3.0169e+00],
        [ 1.4752e+00,  2.4065e-01, -8.5296e-03,  ..., -6.0769e+00,
         -2.5394e-01, -1.5510e+00],

### PyTorch: 新しいautograd関数の定義

In [37]:
import torch

class MyReLU(torch.autograd.Function):
    """
    torch.autograd.Functionをサブクラス化し、Tensors上で動作する順伝播経路と
    逆伝播経路を定義することで、独自のautograd Functionsを実装することができます。
    """
    @staticmethod
    def forward(ctx, input):
        """
        順伝播の経路では入力を含むTensorを受け取り、出力を含むTensorを返します。 
        ctxは逆伝播の際に必要な情報を格納するコンテキストオブジェクトです。
        ctx.save_for_backwardメソッドを使用して、任意のオブジェクトを一時的に保持し、
        逆伝播時に使用することができます。
        """
        ctx.save_for_backward(input)
        return input.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        """
        逆伝播の経路では、出力に対する損失の勾配を含むTensorを受け取り、
        入力に対する損失の勾配を計算する必要があります。
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input


dtype = torch.float
device = torch.device("mps")
# 上の行でTensorFloat32を無効にします。
# TensorFloat32を用いると精度を犠牲にして、ネットワークの高速化を図ることができます。
# https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices

# N：バッチサイズ         D_in：入力層の次元数
# H：隠れ層の次元数       D_out： 出力層の次元数
N, D_in, H, D_out = 64, 1000, 100, 10

# 乱数により入力データと目標となる出力データを表すTensorを生成
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 乱数による重みを表すTensorの定義
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # 関数を適用するには、Function.applyメソッドを用います。
    relu = MyReLU.apply
    # <built-in method apply of FunctionMeta object at 0x1420fc4f0>

    # 順伝播：独自のautograd操作を用いてReLUの出力を算出することで予想結果yを計算します。
    y_pred = relu(x.mm(w1)).mm(w2)

    # 損失の計算と表示
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    loss.backward()

    # 確率的勾配降下法を用いた重みの更新
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # 重みの更新後、手動で勾配を0に初期化
        w1.grad.zero_()
        w2.grad.zero_()

/var/folders/np/chqdmqld4h98fpysyglct_4m0000gp/T/ipykernel_63103/1518199411.py:27: UserWarning: The operator 'aten::nonzero' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1670525849783/work/aten/src/ATen/mps/MPSFallback.mm:11.)
  grad_input[input < 0] = 0


99 748.716552734375
199 6.433542251586914
299 0.11321971565485
399 0.002757652197033167
499 0.00021390017354860902


### nnモジュール

In [38]:
import torch

# N：バッチサイズ         D_in：入力層の次元数
# H：隠れ層の次元数       D_out： 出力層の次元数
N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# nnパッケージを利用し、レイヤーの連なりとしてモデルを定義します。
# nn.Sequentialは他のモジュールを並べて保持することで、それぞれのモジュールを順番に
# 実行し、出力を得ます。各Linearモジュールは線形関数を使用して入力から出力を計算し、
# 重みとバイアスを内部のTensorで保持します。
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for t in range(500):
    y_pred = model(x)

    # 損失の計算と表示
    # 損失関数にyの予測値と正解の値を持つTensorを渡すことで損失を持つTenosrを
    # 得ることができます。
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    model.zero_grad()
    loss.backward()

    # 確率的勾配降下法を用いた重みの更新
    # 各々のパラメータはTensorなので、これまでと同じ方法で勾配を参照することができます。
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

99 2.690983772277832
199 0.039553556591272354
299 0.0013074190355837345
399 6.767728336853907e-05
499 4.4400617298379075e-06


### PyTorch: optim

In [46]:
import torch

# N：バッチサイズ         D_in：入力層の次元数
# H：隠れ層の次元数       D_out： 出力層の次元数
N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# optimパッケージを使用して、モデルの重みを更新するオプティマイザを定義します。
# ここではAdamを使用します。optimパッケージには他にも多くの最適化アルゴリズムが存在ます。
# Adamのコンストラクタの最初の引数により、オプティマイザがどのTensorを更新するか指定できます。
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    y_pred = model(x)

    # 損失の計算と表示
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 逆伝播に入る前に、更新されることになる変数（モデルの学習可能な重み）の勾配を
    # optimaizerを使用して0に初期化します。
    # これは、デフォルトで.backward()が呼び出される度に勾配がバッファに蓄積されるため
    # 必要になる操作です（上書きされるわけではない）。
    # 詳しくはtorch.autograd.backwardのドキュメントを参照してください。
    optimizer.zero_grad()

    # 逆伝播：モデルのパラメータに対応する損失の勾配を計算
    loss.backward()

    # オプティマイザのstep関数を呼び出すことでパラメータを更新
    optimizer.step()

99 51.07844543457031
199 0.8447763919830322
299 0.0031095314770936966
399 8.619559594080783e-06
499 3.07811198752006e-08


In [48]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.0320,  0.0045,  0.0301,  ..., -0.0010, -0.0029,  0.0022],
         [-0.0030, -0.0179, -0.0186,  ..., -0.0099, -0.0111,  0.0371],
         [ 0.0050, -0.0239,  0.0169,  ..., -0.0255, -0.0134, -0.0177],
         ...,
         [ 0.0035, -0.0040, -0.0224,  ...,  0.0293, -0.0314,  0.0327],
         [-0.0207,  0.0050, -0.0247,  ..., -0.0122,  0.0124, -0.0165],
         [ 0.0225, -0.0106,  0.0084,  ..., -0.0049, -0.0001,  0.0035]],
        requires_grad=True),
 Parameter containing:
 tensor([ 0.0414, -0.0166,  0.0062, -0.0093,  0.0158,  0.0222,  0.0095,  0.0297,
          0.0428,  0.0347,  0.0297,  0.0275,  0.0386,  0.0147,  0.0252,  0.0105,
          0.0229,  0.0118,  0.0449, -0.0222,  0.0161,  0.0393,  0.0156, -0.0067,
         -0.0071,  0.0148,  0.0239,  0.0252,  0.0370, -0.0150,  0.0085,  0.0097,
          0.0164, -0.0106,  0.0214, -0.0147, -0.0131,  0.0112,  0.0109,  0.0130,
          0.0431, -0.0171,  0.0323, -0.0020,  0.0028, -0.0156,  0.0203,  0.0236